In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --executor-cores 1 --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark import Row
import json
from pyspark.ml.linalg import Vectors
import pyspark.sql.functions as f
import re
from pyspark.sql.functions import udf
from pyspark.sql.functions import col, rank, lit, split,  udf, desc, when, concat_ws, lower

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

# Train dataframe

In [3]:
schema = StructType() \
      .add("user_id", IntegerType(), True) \
      .add("item_id", IntegerType(), True) \
      .add("purchase", IntegerType(), True) 
      
      
df_user_train = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/labs/slaba03/laba03_train.csv")
df_user_train.show(5)

#     5032624 - len of df_train
#     class balance: 0 - 5021720 / 1 - 10904
#     1941 - unique users
#     3704 - unique items


+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
+-------+-------+--------+
only showing top 5 rows



# Test dataframe

In [4]:
schema = StructType() \
      .add("user_id", IntegerType(), True) \
      .add("item_id", IntegerType(), True) 
      
      
df_user_test = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/labs/slaba03/laba03_test.csv")
df_user_test.show(5)


#     2156840 - len of df_test
#     1941 - unique users
#     3704 - unique items


+-------+-------+
|user_id|item_id|
+-------+-------+
|   1654|  94814|
|   1654|  93629|
|   1654|   9980|
|   1654|  95099|
|   1654|  11265|
+-------+-------+
only showing top 5 rows



# df_items

In [5]:
read_items_schema = StructType(fields=[StructField('item_id', IntegerType()), 
StructField('channel_id', IntegerType()),
StructField('datetime_availability_start', DateType()),
StructField('datetime_availability_stop', DateType()),
StructField('datetime_show_start', DateType()),
StructField('datetime_show_stop', DateType()),
StructField('content_type', IntegerType()),
StructField('title', StringType(), nullable=True),
StructField('year', FloatType(), nullable=True),
StructField('genres', StringType()),
StructField('region_id', IntegerType()),
]) 

# len(df_items) = 635568


df_items = spark.read.format("csv") \
      .option("header", True) \
      .option("sep", "\t")\
      .schema(read_items_schema) \
      .load("/labs/slaba03/laba03_items.csv")

In [6]:
df_items.filter(f.col('content_type')==1).count()

3704

# Изменить

In [7]:
df_items_ = df_items.fillna('Фильмы', subset=['genres']).filter(f.col('content_type')==1).fillna(2005, subset=['year'])

In [8]:
@f.pandas_udf(ArrayType(StringType()))
def reg(object):
    regex = re.compile(u'[\w\d]{2,}', re.U)
    return object.apply(lambda x: regex.findall(x.lower()))
df_items_ = df_items_.withColumn("title_words", reg("title"))\
      .withColumn("genres2", f.split('genres', ','))\
      .drop('channel_id', 'region_id', 'datetime_availability_start').cache()
df_items_.show(5, False, True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------
 item_id                    | 65667                                                                                           
 datetime_availability_stop | 2018-01-01                                                                                      
 datetime_show_start        | null                                                                                            
 datetime_show_stop         | null                                                                                            
 content_type               | 1                                                                                               
 title                      | на пробах только девушки (all girl auditions)                                                   
 year                       | 2013.0                                                                           

In [54]:
def map_func2(object):
    for i, j in genres_replacement.items():
        for k in range(len(object)):
            if i == object[k]:
                object[k] = j
    return object
map_func = f.udf(lambda x : map_func2(x), ArrayType(StringType()))
genres_replacement = {
    'Наши': 'Русские', 'Детские': 'Для детей', 'Драма': 'Драмы', 
    'Биография': 'Биографии', 'Комедия': 'Комедии', 'Боевик': 'Боевики', 
    'Вестерн': 'Вестерны', 'Фантастические': 'Фантастика', 'Исторический': 'Исторические', 
    'Документальный': 'Документальные', 'Мелодрама': 'Мелодрамы', 'Триллер': 'Триллеры', 
    'Военный': 'Военные', ' сказка': 'Сказки', 'Научная фантастика': 'Фантастика', 
    'Анимация': 'Мультфильмы', 'Мультфильм': 'Мультфильмы', 'Музыкальный': 'Музыкальные', 
}

In [55]:
df_items__ = df_items_.filter(f.col('genres').isNotNull()).withColumn('genres3', map_func(f.col('genres2'))).cache()

In [56]:
df_items__1 = df_items__\
.select('item_id', 'year', 'datetime_availability_stop', 
        'datetime_show_start', 'datetime_show_stop', 'content_type',
        'title_words', 'genres3', f.explode('genres3').alias("genres4"))

In [57]:
types = df_items__1.select('genres4').distinct().rdd.flatMap(lambda x: x).collect()

In [58]:
a = []
c = list(types)
for i in c:
    b = "sum(f.col('" + i + "')).alias('" + i + "_')"
    print(b)
    a += [b]

sum(f.col('Ужасы')).alias('Ужасы_')
sum(f.col('Исторические')).alias('Исторические_')
sum(f.col('Мистические')).alias('Мистические_')
sum(f.col('Мультфильмы')).alias('Мультфильмы_')
sum(f.col('Хочу всё знать')).alias('Хочу всё знать_')
sum(f.col('Зарубежные')).alias('Зарубежные_')
sum(f.col('Приключение')).alias('Приключение_')
sum(f.col('Семейный')).alias('Семейный_')
sum(f.col('Союзмультфильм')).alias('Союзмультфильм_')
sum(f.col('Западные мультфильмы')).alias('Западные мультфильмы_')
sum(f.col('Фильмы')).alias('Фильмы_')
sum(f.col('Юмористические')).alias('Юмористические_')
sum(f.col('Детские песни')).alias('Детские песни_')
sum(f.col('Развлекательные')).alias('Развлекательные_')
sum(f.col('Игры')).alias('Игры_')
sum(f.col('Передачи')).alias('Передачи_')
sum(f.col('Короткометражки')).alias('Короткометражки_')
sum(f.col('Русские мультфильмы')).alias('Русские мультфильмы_')
sum(f.col('Боевики')).alias('Боевики_')
sum(f.col('Мелодрамы')).alias('Мелодрамы_')
sum(f.col('Для детей')).alia

In [59]:
a = []
c = list(types)
for i in c:
    b = "sum(f.col('" + i + "')).alias('" + i + "')"
    a += [b]

In [60]:
d = ''
for i in types:
    d += ".fillna(0, subset=['" +  i + "_'])"

In [61]:
d

".fillna(0, subset=['Ужасы_']).fillna(0, subset=['Исторические_']).fillna(0, subset=['Мистические_']).fillna(0, subset=['Мультфильмы_']).fillna(0, subset=['Хочу всё знать_']).fillna(0, subset=['Зарубежные_']).fillna(0, subset=['Приключение_']).fillna(0, subset=['Семейный_']).fillna(0, subset=['Союзмультфильм_']).fillna(0, subset=['Западные мультфильмы_']).fillna(0, subset=['Фильмы_']).fillna(0, subset=['Юмористические_']).fillna(0, subset=['Детские песни_']).fillna(0, subset=['Развлекательные_']).fillna(0, subset=['Игры_']).fillna(0, subset=['Передачи_']).fillna(0, subset=['Короткометражки_']).fillna(0, subset=['Русские мультфильмы_']).fillna(0, subset=['Боевики_']).fillna(0, subset=['Мелодрамы_']).fillna(0, subset=['Для детей_']).fillna(0, subset=['Эротика_']).fillna(0, subset=['Аниме_']).fillna(0, subset=['Вестерны_']).fillna(0, subset=['Короткометражные_']).fillna(0, subset=['Военные_']).fillna(0, subset=['Спортивные_']).fillna(0, subset=['Семейные_']).fillna(0, subset=['Советское к

In [62]:
types = df_items__1.select('genres4').distinct().rdd.flatMap(lambda x: x).collect()
types_expr = [f.when(f.col("genres4") == ty, int(1)).otherwise(int(0)).alias(ty) for ty in types]
df_items_2 = df_items__1.select('genres3', 'year', 'item_id', 'datetime_availability_stop', 
        'datetime_show_start', 'datetime_show_stop', 'content_type',
        'title_words', *types_expr)
df_items_2.show(25, False, True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------
 genres3                    | [Эротика]                                                                                       
 year                       | 2013.0                                                                                          
 item_id                    | 65667                                                                                           
 datetime_availability_stop | 2018-01-01                                                                                      
 datetime_show_start        | null                                                                                            
 datetime_show_stop         | null                                                                                            
 content_type               | 1                                                                                

In [63]:
from pyspark.sql.functions import sum as sum

In [64]:
df_items_3 = df_items_2.groupBy('item_id', 'year', 'datetime_availability_stop', 
        'datetime_show_start', 'datetime_show_stop', 'content_type', 'title_words')\
.agg(sum(f.col('Ужасы')).alias('Ужасы'), sum(f.col('Исторические')).alias('Исторические'), 
     sum(f.col('Мистические')).alias('Мистические'), sum(f.col('Мультфильмы')).alias('Мультфильмы'), 
     sum(f.col('Хочу всё знать')).alias('Хочу всё знать'), sum(f.col('Зарубежные')).alias('Зарубежные'), 
     sum(f.col('Приключение')).alias('Приключение'), sum(f.col('Семейный')).alias('Семейный'), 
     sum(f.col('Союзмультфильм')).alias('Союзмультфильм'), sum(f.col('Западные мультфильмы')).alias('Западные мультфильмы'), 
     sum(f.col('Фильмы')).alias('Фильмы'), sum(f.col('Юмористические')).alias('Юмористические'), 
     sum(f.col('Детские песни')).alias('Детские песни'), sum(f.col('Развлекательные')).alias('Развлекательные'), 
     sum(f.col('Игры')).alias('Игры'), sum(f.col('Передачи')).alias('Передачи'), 
     sum(f.col('Короткометражки')).alias('Короткометражки'), sum(f.col('Русские мультфильмы')).alias('Русские мультфильмы'), 
     sum(f.col('Боевики')).alias('Боевики'), sum(f.col('Мелодрамы')).alias('Мелодрамы'), 
     sum(f.col('Для детей')).alias('Для детей'), sum(f.col('Эротика')).alias('Эротика'), 
     sum(f.col('Аниме')).alias('Аниме'), sum(f.col('Вестерны')).alias('Вестерны'), 
     sum(f.col('Короткометражные')).alias('Короткометражные'), sum(f.col('Военные')).alias('Военные'), 
     sum(f.col('Спортивные')).alias('Спортивные'), sum(f.col('Семейные')).alias('Семейные'), 
     sum(f.col('Советское кино')).alias('Советское кино'), sum(f.col('Драмы')).alias('Драмы'), 
     sum(f.col('Криминал')).alias('Криминал'), sum(f.col('О здоровье')).alias('О здоровье'), 
     sum(f.col('Мюзиклы')).alias('Мюзиклы'), sum(f.col('Для взрослых')).alias('Для взрослых'), 
     sum(f.col('Приключения')).alias('Приключения'), sum(f.col('Фильмы в 3D')).alias('Фильмы в 3D'), 
     sum(f.col('Романтические')).alias('Романтические'), sum(f.col('Познавательные')).alias('Познавательные'), 
     sum(f.col('Спорт')).alias('Спорт'), sum(f.col('Охота и рыбалка')).alias('Охота и рыбалка'), 
     sum(f.col('Фильмы-спектакли')).alias('Фильмы-спектакли'), sum(f.col('Комедии')).alias('Комедии'), 
     sum(f.col('Полнометражные')).alias('Полнометражные'), sum(f.col('Сериалы')).alias('Сериалы'), 
     sum(f.col('Для всей семьи')).alias('Для всей семьи'), sum(f.col('Мультфильмы в 3D')).alias('Мультфильмы в 3D'), 
     sum(f.col('Экранизации')).alias('Экранизации'), sum(f.col('Документальные')).alias('Документальные'), 
     sum(f.col('Арт-хаус')).alias('Арт-хаус'), sum(f.col('Для самых маленьких')).alias('Для самых маленьких'), 
     sum(f.col('Развивающие')).alias('Развивающие'), sum(f.col('Фантастика')).alias('Фантастика'), 
     sum(f.col('Сказки')).alias('Сказки'), sum(f.col('Мультсериалы')).alias('Мультсериалы'), 
     sum(f.col('Фэнтези')).alias('Фэнтези'), sum(f.col('Русские')).alias('Русские'), 
     sum(f.col('Про животных')).alias('Про животных'), sum(f.col('Кулинария')).alias('Кулинария'), 
     sum(f.col('Музыкальные')).alias('Музыкальные'), sum(f.col('Советские')).alias('Советские'), 
     sum(f.col('Артхаус')).alias('Артхаус'), sum(f.col('Биографии')).alias('Биографии'), 
     sum(f.col('Детективы')).alias('Детективы'), sum(f.col('Реалити-шоу')).alias('Реалити-шоу'), 
     sum(f.col('Триллеры')).alias('Триллеры'), sum(f.col('Видеоигры')).alias('Видеоигры'), 
     sum(f.col('Прочие')).alias('Прочие')).cache()

In [ ]:
df_items_3.show()

In [21]:
item_activities=df_user_train.join(df_items_3, on='item_id')\
.groupBy('item_id')\
.agg(sum(f.col('purchase')).alias('purchase_item'))

In [22]:
user_activities=df_user_train.join(df_items_3, on='item_id').filter(f.col('purchase') == 1)\
.groupBy('user_id')\
.agg(sum(f.col('purchase')).alias('purchase_all'), sum(f.col('Ужасы')).alias('Ужасы_'), sum(f.col('Исторические')).alias('Исторические_'), 
     sum(f.col('Мистические')).alias('Мистические_'), sum(f.col('Мультфильмы')).alias('Мультфильмы_'), 
     sum(f.col('Хочу всё знать')).alias('Хочу всё знать_'), sum(f.col('Зарубежные')).alias('Зарубежные_'), 
     sum(f.col('Приключение')).alias('Приключение_'), sum(f.col('Семейный')).alias('Семейный_'), 
     sum(f.col('Союзмультфильм')).alias('Союзмультфильм_'), sum(f.col('Западные мультфильмы')).alias('Западные мультфильмы_'), 
     sum(f.col('Фильмы')).alias('Фильмы_'), sum(f.col('Юмористические')).alias('Юмористические_'), 
     sum(f.col('Детские песни')).alias('Детские песни_'), sum(f.col('Развлекательные')).alias('Развлекательные_'), 
     sum(f.col('Игры')).alias('Игры_'), sum(f.col('Передачи')).alias('Передачи_'), 
     sum(f.col('Короткометражки')).alias('Короткометражки_'), sum(f.col('Русские мультфильмы')).alias('Русские мультфильмы_'), 
     sum(f.col('Боевики')).alias('Боевики_'), sum(f.col('Мелодрамы')).alias('Мелодрамы_'), 
     sum(f.col('Для детей')).alias('Для детей_'), sum(f.col('Эротика')).alias('Эротика_'), 
     sum(f.col('Аниме')).alias('Аниме_'), sum(f.col('Вестерны')).alias('Вестерны_'), 
     sum(f.col('Короткометражные')).alias('Короткометражные_'), sum(f.col('Военные')).alias('Военные_'), 
     sum(f.col('Спортивные')).alias('Спортивные_'), sum(f.col('Семейные')).alias('Семейные_'), 
     sum(f.col('Советское кино')).alias('Советское кино_'), sum(f.col('Драмы')).alias('Драмы_'), 
     sum(f.col('Криминал')).alias('Криминал_'), sum(f.col('О здоровье')).alias('О здоровье_'), 
     sum(f.col('Мюзиклы')).alias('Мюзиклы_'), sum(f.col('Для взрослых')).alias('Для взрослых_'), 
     sum(f.col('Приключения')).alias('Приключения_'), sum(f.col('Фильмы в 3D')).alias('Фильмы в 3D_'), 
     sum(f.col('Романтические')).alias('Романтические_'), sum(f.col('Познавательные')).alias('Познавательные_'), 
     sum(f.col('Спорт')).alias('Спорт_'), sum(f.col('Охота и рыбалка')).alias('Охота и рыбалка_'), 
     sum(f.col('Фильмы-спектакли')).alias('Фильмы-спектакли_'), sum(f.col('Комедии')).alias('Комедии_'), 
     sum(f.col('Полнометражные')).alias('Полнометражные_'), sum(f.col('Сериалы')).alias('Сериалы_'), 
     sum(f.col('Для всей семьи')).alias('Для всей семьи_'), sum(f.col('Мультфильмы в 3D')).alias('Мультфильмы в 3D_'), 
     sum(f.col('Экранизации')).alias('Экранизации_'), sum(f.col('Документальные')).alias('Документальные_'), 
     sum(f.col('Арт-хаус')).alias('Арт-хаус_'), sum(f.col('Для самых маленьких')).alias('Для самых маленьких_'), 
     sum(f.col('Развивающие')).alias('Развивающие_'), sum(f.col('Фантастика')).alias('Фантастика_'), 
     sum(f.col('Сказки')).alias('Сказки_'), sum(f.col('Мультсериалы')).alias('Мультсериалы_'), 
     sum(f.col('Фэнтези')).alias('Фэнтези_'), sum(f.col('Русские')).alias('Русские_'), 
     sum(f.col('Про животных')).alias('Про животных_'), sum(f.col('Кулинария')).alias('Кулинария_'), 
     sum(f.col('Музыкальные')).alias('Музыкальные_'), sum(f.col('Советские')).alias('Советские_'), 
     sum(f.col('Артхаус')).alias('Артхаус_'), sum(f.col('Биографии')).alias('Биографии_'), 
     sum(f.col('Детективы')).alias('Детективы_'), sum(f.col('Реалити-шоу')).alias('Реалити-шоу_'), 
     sum(f.col('Триллеры')).alias('Триллеры_'), sum(f.col('Видеоигры')).alias('Видеоигры_'), 
     sum(f.col('Прочие')).alias('Прочие_'))\
.withColumnRenamed('user_id', 'user_id2')

In [23]:
item_activities = item_activities.join(df_items_3, on='item_id')

In [24]:
df_user_train2=df_user_train.join(item_activities, on='item_id')
df_user_train2 = df_user_train2.join(user_activities, df_user_train2.user_id == user_activities.user_id2, 'leftouter')\
.drop('user_id2').cache()

In [ ]:
df_user_train2.count()

In [25]:
df_user_train2 = df_user_train2.fillna(0, subset=['Ужасы_'])\
.fillna(0, subset=['Исторические_']).fillna(0, subset=['Мистические_']).fillna(0, subset=['Мультфильмы_'])\
.fillna(0, subset=['Хочу всё знать_']).fillna(0, subset=['Зарубежные_']).fillna(0, subset=['Приключение_'])\
.fillna(0, subset=['Семейный_']).fillna(0, subset=['Союзмультфильм_']).fillna(0, subset=['Западные мультфильмы_'])\
.fillna(0, subset=['Фильмы_']).fillna(0, subset=['Юмористические_']).fillna(0, subset=['Детские песни_'])\
.fillna(0, subset=['Развлекательные_']).fillna(0, subset=['Игры_']).fillna(0, subset=['Передачи_'])\
.fillna(0, subset=['Короткометражки_']).fillna(0, subset=['Русские мультфильмы_']).fillna(0, subset=['Боевики_'])\
.fillna(0, subset=['Мелодрамы_']).fillna(0, subset=['Для детей_']).fillna(0, subset=['Эротика_'])\
.fillna(0, subset=['Аниме_']).fillna(0, subset=['Вестерны_']).fillna(0, subset=['Короткометражные_'])\
.fillna(0, subset=['Военные_']).fillna(0, subset=['Спортивные_']).fillna(0, subset=['Семейные_'])\
.fillna(0, subset=['Советское кино_']).fillna(0, subset=['Драмы_']).fillna(0, subset=['Криминал_'])\
.fillna(0, subset=['О здоровье_']).fillna(0, subset=['Мюзиклы_']).fillna(0, subset=['Для взрослых_'])\
.fillna(0, subset=['Приключения_']).fillna(0, subset=['Фильмы в 3D_']).fillna(0, subset=['Романтические_'])\
.fillna(0, subset=['Познавательные_']).fillna(0, subset=['Спорт_']).fillna(0, subset=['Охота и рыбалка_'])\
.fillna(0, subset=['Фильмы-спектакли_']).fillna(0, subset=['Комедии_']).fillna(0, subset=['Полнометражные_'])\
.fillna(0, subset=['Сериалы_']).fillna(0, subset=['Для всей семьи_']).fillna(0, subset=['Мультфильмы в 3D_'])\
.fillna(0, subset=['Экранизации_']).fillna(0, subset=['Документальные_']).fillna(0, subset=['Арт-хаус_'])\
.fillna(0, subset=['Для самых маленьких_']).fillna(0, subset=['Развивающие_']).fillna(0, subset=['Фантастика_'])\
.fillna(0, subset=['Сказки_']).fillna(0, subset=['Мультсериалы_']).fillna(0, subset=['Фэнтези_'])\
.fillna(0, subset=['Русские_']).fillna(0, subset=['Про животных_']).fillna(0, subset=['Кулинария_'])\
.fillna(0, subset=['Музыкальные_']).fillna(0, subset=['Советские_']).fillna(0, subset=['Артхаус_'])\
.fillna(0, subset=['Биографии_']).fillna(0, subset=['Детективы_']).fillna(0, subset=['Реалити-шоу_'])\
.fillna(0, subset=['Триллеры_']).fillna(0, subset=['Видеоигры_']).fillna(0, subset=['Прочие_']).fillna(0, subset=['purchase_all'])

In [26]:
df_user_train2.cache().show(2, False, True)

-RECORD 0-------------------------------------
 item_id                    | 5117            
 user_id                    | 754230          
 purchase                   | 0               
 purchase_item              | 2               
 year                       | 2011.0          
 datetime_availability_stop | 2099-12-31      
 datetime_show_start        | null            
 datetime_show_stop         | null            
 content_type               | 1               
 title_words                | [конан, варвар] 
 Ужасы                      | 0               
 Исторические               | 0               
 Мистические                | 0               
 Мультфильмы                | 0               
 Хочу всё знать             | 0               
 Зарубежные                 | 1               
 Приключение                | 0               
 Семейный                   | 0               
 Союзмультфильм             | 0               
 Западные мультфильмы       | 0               
 Фильмы      

In [27]:
dataSet=df_user_train2.withColumn('id',concat_ws('_',col('item_id'),col('user_id'))).drop('item_id','user_id').cache()

In [28]:
dataSet

DataFrame[purchase: int, purchase_item: bigint, year: float, datetime_availability_stop: date, datetime_show_start: date, datetime_show_stop: date, content_type: int, title_words: array<string>, Ужасы: bigint, Исторические: bigint, Мистические: bigint, Мультфильмы: bigint, Хочу всё знать: bigint, Зарубежные: bigint, Приключение: bigint, Семейный: bigint, Союзмультфильм: bigint, Западные мультфильмы: bigint, Фильмы: bigint, Юмористические: bigint, Детские песни: bigint, Развлекательные: bigint, Игры: bigint, Передачи: bigint, Короткометражки: bigint, Русские мультфильмы: bigint, Боевики: bigint, Мелодрамы: bigint, Для детей: bigint, Эротика: bigint, Аниме: bigint, Вестерны: bigint, Короткометражные: bigint, Военные: bigint, Спортивные: bigint, Семейные: bigint, Советское кино: bigint, Драмы: bigint, Криминал: bigint, О здоровье: bigint, Мюзиклы: bigint, Для взрослых: bigint, Приключения: bigint, Фильмы в 3D: bigint, Романтические: bigint, Познавательные: bigint, Спорт: bigint, Охота и р

In [42]:
unique_genres = df_items__.filter(f.col('content_type')==1).select(f.explode('genres3').alias('unique_genres'))\
.groupBy('unique_genres')\
.count().orderBy(f.col('count').desc()).show(200)

+--------------------+-----+
|       unique_genres|count|
+--------------------+-----+
|          Зарубежные| 1740|
|             Русские| 1099|
|               Драмы|  991|
|             Комедии|  875|
|            Триллеры|  660|
|           Для детей|  559|
|             Боевики|  555|
|           Мелодрамы|  479|
|         Приключения|  437|
|               Ужасы|  384|
|           Детективы|  355|
|          Фантастика|  353|
|            Криминал|  272|
|         Мультфильмы|  244|
|      Для всей семьи|  222|
|             Военные|  179|
| Русские мультфильмы|  169|
|      Полнометражные|  161|
|        Исторические|  125|
|             Эротика|  124|
|           Советские|  123|
|      Союзмультфильм|  118|
|Западные мультфильмы|  118|
|      Советское кино|  117|
|             Сериалы|  116|
|      Документальные|  109|
|        Про животных|   95|
|             Фэнтези|   81|
|              Фильмы|   80|
|         Мистические|   72|
|            Семейные|   72|
|        Мульт

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['purchase_all', 'purchase_item', 'year', "Ужасы", "Исторические", 
                                       "Мистические", "Мультфильмы", "Хочу всё знать", "Зарубежные", "Приключение", 
                                       "Семейный", "Союзмультфильм", "Западные мультфильмы", "Фильмы", "Юмористические", 
                                       "Детские песни", "Развлекательные", "Игры", "Передачи", "Короткометражки", 
                                       "Русские мультфильмы", "Боевики", "Мелодрамы", "Для детей", "Эротика", "Аниме", 
                                       "Вестерны", "Короткометражные", "Военные", "Спортивные", "Семейные", 
                                       "Советское кино", "Драмы", "Криминал", "Мюзиклы", "Для взрослых", 
                                       "Приключения", "Романтические", "Познавательные", "Спорт", 
                                       "Охота и рыбалка", "Комедии", "Полнометражные", "Сериалы", 
                                       "Для всей семьи", "Экранизации", "Документальные", "Арт-хаус", 
                                       "Для самых маленьких", "Развивающие", "Фантастика", "Сказки", "Мультсериалы", 
                                       "Фэнтези", "Русские", "Про животных", "Музыкальные", "Советские", 
                                       "Артхаус", "Биографии", "Детективы", "Реалити-шоу", "Триллеры", 
                                       "Видеоигры", "Прочие", "Ужасы_", "Исторические_", "Мистические_", "Мультфильмы_", 
                                       "Хочу всё знать_", "Зарубежные_", "Приключение_", "Семейный_", "Союзмультфильм_", 
                                       "Западные мультфильмы_", "Фильмы_", "Юмористические_", "Детские песни_", 
                                       "Развлекательные_", "Игры_", "Передачи_", "Короткометражки_", "Русские мультфильмы_", 
                                       "Боевики_", "Мелодрамы_", "Для детей_", "Эротика_", "Аниме_", "Вестерны_", 
                                       "Короткометражные_", "Военные_", "Спортивные_", "Семейные_", "Советское кино_", "Драмы_", 
                                       "Криминал_", "Мюзиклы_", "Для взрослых_", "Приключения_",  
                                       "Романтические_", "Познавательные_", "Спорт_", "Охота и рыбалка_",  
                                       "Комедии_", "Полнометражные_", "Сериалы_", "Для всей семьи_", "Экранизации_", 
                                       "Документальные_", "Арт-хаус_", "Для самых маленьких_", "Развивающие_", "Фантастика_", 
                                       "Сказки_", "Мультсериалы_", "Фэнтези_", "Русские_", "Про животных_",  
                                       "Музыкальные_", "Советские_", "Артхаус_", "Биографии_", "Детективы_", "Реалити-шоу_", 
                                       "Триллеры_", "Видеоигры_", "Прочие_"], outputCol="features")
dataSet3=assembler.transform(dataSet).select('id','purchase','features').withColumnRenamed('purchase', 'label')
dataSet3.show(2)

In [30]:
dataSet3

DataFrame[id: string, label: int, features: vector]

In [ ]:
train, test22 = dataSet3.randomSplit([0.7, 0.3], seed = 2018)

In [31]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [34]:
gb = GBTClassifier(featuresCol='features', labelCol='label', maxDepth=4, maxBins=32, maxIter = 20)
gbparamGrid = (ParamGridBuilder()
             .addGrid(gb.maxDepth, [2, 5, 7])
             .addGrid(gb.maxBins, [10, 20, 40])
             .addGrid(gb.maxIter, [5, 10, 20])
             .build())

In [36]:
gb = GBTClassifier(featuresCol='features', labelCol='label', maxDepth=4, maxBins=32, maxIter = 30)

In [35]:
gbevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
gbcv = CrossValidator(estimator = gb,
                      estimatorParamMaps = gbparamGrid,
                      evaluator = gbevaluator,
                      numFolds = 5)

In [ ]:
train

# запуск

In [37]:
%%time
gbt_model = gb.fit(dataSet3)
#dataSet3

CPU times: user 1.25 s, sys: 1.07 s, total: 2.33 s
Wall time: 13min 41s


In [38]:
test_df=df_user_test.join(item_activities,on='item_id')
test_df = test_df.join(user_activities, test_df.user_id == user_activities.user_id2, 'leftouter')\
.drop('user_id2').cache()

In [39]:
test_df = test_df.fillna(0, subset=['Ужасы_'])\
.fillna(0, subset=['Исторические_']).fillna(0, subset=['Мистические_']).fillna(0, subset=['Мультфильмы_'])\
.fillna(0, subset=['Хочу всё знать_']).fillna(0, subset=['Зарубежные_']).fillna(0, subset=['Приключение_'])\
.fillna(0, subset=['Семейный_']).fillna(0, subset=['Союзмультфильм_']).fillna(0, subset=['Западные мультфильмы_'])\
.fillna(0, subset=['Фильмы_']).fillna(0, subset=['Юмористические_']).fillna(0, subset=['Детские песни_'])\
.fillna(0, subset=['Развлекательные_']).fillna(0, subset=['Игры_']).fillna(0, subset=['Передачи_'])\
.fillna(0, subset=['Короткометражки_']).fillna(0, subset=['Русские мультфильмы_']).fillna(0, subset=['Боевики_'])\
.fillna(0, subset=['Мелодрамы_']).fillna(0, subset=['Для детей_']).fillna(0, subset=['Эротика_'])\
.fillna(0, subset=['Аниме_']).fillna(0, subset=['Вестерны_']).fillna(0, subset=['Короткометражные_'])\
.fillna(0, subset=['Военные_']).fillna(0, subset=['Спортивные_']).fillna(0, subset=['Семейные_'])\
.fillna(0, subset=['Советское кино_']).fillna(0, subset=['Драмы_']).fillna(0, subset=['Криминал_'])\
.fillna(0, subset=['О здоровье_']).fillna(0, subset=['Мюзиклы_']).fillna(0, subset=['Для взрослых_'])\
.fillna(0, subset=['Приключения_']).fillna(0, subset=['Фильмы в 3D_']).fillna(0, subset=['Романтические_'])\
.fillna(0, subset=['Познавательные_']).fillna(0, subset=['Спорт_']).fillna(0, subset=['Охота и рыбалка_'])\
.fillna(0, subset=['Фильмы-спектакли_']).fillna(0, subset=['Комедии_']).fillna(0, subset=['Полнометражные_'])\
.fillna(0, subset=['Сериалы_']).fillna(0, subset=['Для всей семьи_']).fillna(0, subset=['Мультфильмы в 3D_'])\
.fillna(0, subset=['Экранизации_']).fillna(0, subset=['Документальные_']).fillna(0, subset=['Арт-хаус_'])\
.fillna(0, subset=['Для самых маленьких_']).fillna(0, subset=['Развивающие_']).fillna(0, subset=['Фантастика_'])\
.fillna(0, subset=['Сказки_']).fillna(0, subset=['Мультсериалы_']).fillna(0, subset=['Фэнтези_'])\
.fillna(0, subset=['Русские_']).fillna(0, subset=['Про животных_']).fillna(0, subset=['Кулинария_'])\
.fillna(0, subset=['Музыкальные_']).fillna(0, subset=['Советские_']).fillna(0, subset=['Артхаус_'])\
.fillna(0, subset=['Биографии_']).fillna(0, subset=['Детективы_']).fillna(0, subset=['Реалити-шоу_'])\
.fillna(0, subset=['Триллеры_']).fillna(0, subset=['Видеоигры_']).fillna(0, subset=['Прочие_']).fillna(0, subset=['purchase_all'])

In [40]:
test=test_df\
.withColumn('id',concat_ws('_',col('item_id'),col('user_id')))\
.drop('item_id','user_id').cache()

test=assembler.transform(test).select('id','features')
test.show(2)

+------------+--------------------+
|          id|            features|
+------------+--------------------+
|11025_754230|(137,[0,2,8,21,37...|
|72912_754230|(137,[0,2,44,58,7...|
+------------+--------------------+
only showing top 2 rows



In [41]:
predictions = gbt_model.transform(test)
def get_result_(array):
    return float(array[1])
get_result_udf = udf(get_result_,DoubleType())
pred_= predictions\
.withColumn('purchase',get_result_udf(col('probability')))\
.withColumn('item_id',split(col('id'),'_')[0].cast('int'))\
.withColumn('user_id',split(col('id'),'_')[1].cast('int'))\
.select('user_id','item_id','purchase')\
.orderBy('user_id','item_id')
pred_.toPandas().to_csv('lab03.csv')

In [ ]:
predictions.show(10, False, True)

In [ ]:
pred_

In [ ]:
pred_.show()

In [ ]:
pred_.filter(f.col('purchase') > 0.05).orderBy(f.col('purchase').desk()).show()

In [ ]:
pred_

In [ ]:
0.08315
0.05484
0.04879


# Views df

In [ ]:
read_users_schema = StructType(fields=[StructField('user_id', IntegerType()), 
StructField('item_id', IntegerType()),
StructField('ts_start', IntegerType()),
StructField('ts_end', IntegerType()),
StructField('item_type', StringType()),
]) 

# count = 20845607
# user_id = 79385
# item_id = 633840
#     class balance: live - 17704201 / pvr - 3141406


df_views_programmes = spark.read.format("csv") \
      .option("header", True) \
      .schema(read_users_schema) \
      .load("/labs/slaba03/laba03_views_programmes.csv")
df_views_programmes = df_views_programmes.withColumn('ts_all', f.col('ts_end').cast('long') - f.col('ts_start').cast('long'))
df_views_programmes.show(5, False, False)

In [ ]:
df_user_test.join(df_views_programmes, on = 'item_id').count()

In [ ]:
df_user_test

In [ ]:
df_views_programmes

In [ ]:
write_kafka_params = {
   "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
   "topic": "ivan.ivanov"
}
batch_df.writeStream.format("kafka").options(**write_kafka_params)\
    .option("checkpointLocation", "streaming/chk/chk_kafka")\
    .outputMode("append").start()

In [ ]:
spark.stop()